# Simple Cycle Examples with BMS and DARTS
The aim of this example notebook is to use the AutoRA `Cycle` to recover a simple ground truth theory from some noisy data using BSM and DARTS, as a proof of concept.
It uses a trivial experimentalist which resamples the same x-values each cycle.

In [ ]:
# Uncomment the following line when running on Google Colab
# !pip install autora

In [ ]:
from autora.variable import VariableCollection, Variable
from autora.cycle import Cycle, plot_results_panel_2d
from itertools import repeat, chain

In [ ]:
def ground_truth(xs):
    return (xs ** 2.) + xs + 1.

The space of allowed x values is the integers between 0 and 10 inclusive, and we record the allowed output values as well.

In [ ]:
study_metadata = VariableCollection(
   independent_variables=[Variable(name="x1", allowed_values=range(11))],
   dependent_variables=[Variable(name="y", value_range=(-20, 20))],
   )

The experimentalist is used to propose experiments.
Since the space of values is so restricted, we can just sample them all each time.

In [ ]:
from autora.experimentalist.pipeline import make_pipeline
example_experimentalist = make_pipeline(
    [list(chain.from_iterable((repeat(study_metadata.independent_variables[0].allowed_values, 10))))])

When we run a synthetic experiment, we get a reproducible noisy result:

In [ ]:
import numpy as np

def get_example_synthetic_experiment_runner():
    rng = np.random.default_rng(seed=180)
    def runner(xs):
        return ground_truth(xs) + rng.normal(0, 1.0, xs.shape)
    return runner

example_synthetic_experiment_runner = get_example_synthetic_experiment_runner()
x = np.array([1.])
example_synthetic_experiment_runner(x)

## Bayesian Machine Scientist

In [ ]:
from autora.skl.bms import BMSRegressor
bms_theorist = BMSRegressor(epochs=100)

We initialize the Cycle with the metadata describing the domain of the theory,
the theorist, experimentalist and experiment runner,
as well as a monitor which will let us know which cycle we're currently on.

In [ ]:
cycle = Cycle(
    metadata=study_metadata,
    theorist=bms_theorist,
    experimentalist=example_experimentalist,
    experiment_runner=example_synthetic_experiment_runner
)

We can run the cycle by calling the run method:

In [ ]:
cycle.run(num_cycles=3)

We can now interrogate the results. The first set of conditions which went into the
experiment runner were:

The observations include the conditions and the results:

In [ ]:
cycle.data.observations[0]

The best fit theory after the first cycle is:

In [ ]:
len(cycle.data.observations)

In [ ]:
str(cycle.data.theories[0].model_), cycle.data.theories[0].model_.fit_par[str(cycle.data.theories[0].model_)]

In [ ]:
str(cycle.data.theories[-1].model_), cycle.data.theories[-1].model_.fit_par[str(cycle.data.theories[-1].model_)]

In [ ]:
# Plot all cycle results
plot_results_panel_2d(cycle, subplot_kw=dict(figsize=(12,4)))

## DARTS


In [ ]:
from autora.skl.darts import DARTSRegressor
darts_theorist = DARTSRegressor(max_epochs=100)

In [ ]:
darts_cycle = Cycle(
    metadata=study_metadata,
    theorist=darts_theorist,
    experimentalist=example_experimentalist,
    experiment_runner=example_synthetic_experiment_runner
)

In [ ]:
darts_cycle.run(3)

In [ ]:
darts_cycle.data.theories[-2].visualize_model()


In [ ]:
darts_cycle.data.theories[-2].model_repr()


In [ ]:
# Rerun 3 more times
darts_cycle.run(3)

In [ ]:
# Plot the all cycle results
plot_results_panel_2d(darts_cycle, wrap=3)
